# <center> 2차 크롤링 - <원티드> </center>

##### 링크들 들어가서 각각 정보 수집, 자율주행

In [1]:
from tqdm.notebook import tqdm
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
import time
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.request import Request
import urllib


In [87]:
options = webdriver.ChromeOptions()

url = "https://www.wanted.co.kr/jobsfeed"
driver = webdriver.Chrome(service=Service("../../driver/chromedriver"),
                          options=options)

driver.get(url)

In [88]:
driver.get(url)

In [68]:
nav_element = driver.find_element(By.CLASS_NAME, "Aside_asideList__3WNyu")
search_element = nav_element.find_element(By.CLASS_NAME, "SvgIcon_SvgIcon__root__8vwon")

search_element.click()

In [69]:
search_doc = driver.find_element(By.CLASS_NAME, "SearchInput_SearchInput__gySrv")
search_doc.send_keys('자율주행')
search_doc.send_keys(Keys.ENTER)


In [70]:
wait = WebDriverWait(driver, 10)
position_tab = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[contains(text(), "포지션")]')))
position_tab.click()

In [71]:
#서울 경기 구분용
simple_list = []
wait = WebDriverWait(driver, 10)  
simple_name_elements = []  

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    try:
        wait.until(lambda driver: len(driver.find_elements(By.CLASS_NAME, 'JobCard_location__2EOr5')) > len(simple_name_elements))
    except TimeoutException:
        break

    new_elements = driver.find_elements(By.CLASS_NAME, 'JobCard_location__2EOr5')
    simple_name_elements.extend(new_elements)

    # StaleElementReferenceException을 처리하기 위해 엘리먼트 수의 변경을 확인
    try:
        wait.until(lambda driver: len(driver.find_elements(By.CLASS_NAME, 'JobCard_location__2EOr5')) > len(simple_name_elements))
    except TimeoutException:
        break

    if len(simple_name_elements) >= 20:
        break

# 기업 이름 추출 및 출력
simple_names = [element.text.strip() for element in simple_name_elements]
print(simple_names)
print(len(simple_names))

['', '', '', '', '', '', '', '', '서울 · 한국', '서울 · 한국', '경기 · 한국', '서울 · 한국', '경기 · 한국', '경북 · 한국', '서울 · 한국', '서울 · 한국', '경기 · 한국', '경기 · 한국', '서울 · 한국', '서울 · 한국', '경기 · 한국', '경기 · 한국', '경기 · 한국', '경기 · 한국', '서울 · 한국', '경기 · 한국', '서울 · 한국', '서울 · 한국', '서울 · 한국', '경기 · 한국', '경기 · 한국']
31


In [72]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import time

# 기업이름 요소 초기 로드
company_name_elements = driver.find_elements(By.CLASS_NAME, 'JobCard_companyName__vZMqJ')
num_elements_before_scroll = len(company_name_elements)

while True:
    # 페이지 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2) 

    try:
        # 새로운 회사명 요소들이 로드될 때까지 대기 (타임아웃 10초)
        wait.until(lambda driver: len(driver.find_elements(By.CLASS_NAME, 'JobCard_companyName__vZMqJ')) > num_elements_before_scroll)
    except TimeoutException as ex:
        break

    # 현재 로드된 회사명 요소들 가져오기
    company_name_elements = driver.find_elements(By.CLASS_NAME, 'JobCard_companyName__vZMqJ')
    num_elements_before_scroll = len(company_name_elements)

    if num_elements_before_scroll >= 20:  
        break

company_names = []
href_lists = []

href_elements = driver.find_elements(By.CSS_SELECTOR, 'a[data-attribute-id="position__click"]')

# 가져온 데이터 중에서 조건에 맞는 것들만 필터링
for i, element in enumerate(company_name_elements):
    simple_name = simple_names[min(i, len(simple_names) - 1)]
    company_name_text = element.text.strip()


    # simple_name에 '서울 · 한국' 또는 '경기 · 한국'이 포함된 경우에만 추가
    if '서울 · 한국' in simple_name or '경기 · 한국' in simple_name:
        company_names.append(company_name_text)
        href_value = href_elements[i].get_attribute('href')
        href_lists.append(href_value)


In [73]:
print(company_names)
print(href_lists)
print(len(company_names))
print(len(href_lists))

['컨트롤웍스', '뷰런테크놀로지', '씨엔비스', '컨트롤웍스', '베스텔라랩', '엑스와이지', '마스오토', '슈어소프트테크', '에스티피씨', '알지티', '컨트롤웍스', '슈어소프트테크', '에이모(AIMMO)', '클로봇', '긴트', '뷰런테크놀로지', '에스티피씨', '포티투닷(42dot)', '포티투닷(42dot)', '포티투닷(42dot)', '에스티피씨', '에스티피씨']
['https://www.wanted.co.kr/wd/190893', 'https://www.wanted.co.kr/wd/199355', 'https://www.wanted.co.kr/wd/196842', 'https://www.wanted.co.kr/wd/190898', 'https://www.wanted.co.kr/wd/111438', 'https://www.wanted.co.kr/wd/199578', 'https://www.wanted.co.kr/wd/198840', 'https://www.wanted.co.kr/wd/198898', 'https://www.wanted.co.kr/wd/189189', 'https://www.wanted.co.kr/wd/182743', 'https://www.wanted.co.kr/wd/190899', 'https://www.wanted.co.kr/wd/198922', 'https://www.wanted.co.kr/wd/132158', 'https://www.wanted.co.kr/wd/162533', 'https://www.wanted.co.kr/wd/141290', 'https://www.wanted.co.kr/wd/199350', 'https://www.wanted.co.kr/wd/169773', 'https://www.wanted.co.kr/wd/150909', 'https://www.wanted.co.kr/wd/118630', 'https://www.wanted.co.kr/wd/140554', 'https://www.wante

In [74]:
#직무(담당업무 열)가져오기
job_title_elements = driver.find_elements(By.CLASS_NAME, 'JobCard_title__ddkwM') 
num_elements_before_scroll = len(company_name_elements)

while True:
    # 페이지 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2) 

    try:
        # 새로운 회사명 요소들이 로드될 때까지 대기 (타임아웃 10초)
        wait.until(lambda driver: len(driver.find_elements(By.CLASS_NAME, 'JobCard_title__ddkwM')) > num_elements_before_scroll)
    except TimeoutException as ex:
        break

    job_title_elements = driver.find_elements(By.CLASS_NAME, 'JobCard_title__ddkwM')
    num_elements_before_scroll = len(company_name_elements)

    if num_elements_before_scroll >= 20:  
        break

job_titles = []

for i, element in enumerate(job_title_elements):
    # simple_names 리스트에서 현재 인덱스에 해당하는 값 가져오기
    simple_name = simple_names[min(i, len(simple_names)-1)]

    # 만약 값이 '서울'이나 '경기'로 시작하면 추가
    if simple_name.startswith('서울') or simple_name.startswith('경기'):
        job_titles.append(job_title_elements[i].text.strip())  # 수정된 부분



In [75]:

print("Job Titles:", job_titles)
print(len(job_titles))

Job Titles: ['[2년 이상] 자율주행 Front-End 개발자', '자율주행 인지 개발자', '자율주행 인지 개발자', '자율주행 SW 엔지니어', '자율주행 연구원(경기/대구)(전문연구요원가능)', '자율주행 로봇 엔지니어', '소프트웨어 엔지니어 (자율주행)', '[자율주행검증개발팀] C++ 개발자', '[신입] 자율주행 Perception Engineer', '자율주행 F/W엔지니어', '자율주행 플랫폼 엔지니어', '[자율주행기술팀] SW 엔지니어링 (개발 및 검증)', '자율주행 System Engineer (SE)', '자율주행 개발자 (병특지원가능)', '자율주행 개발자', '자율주행 판단/제어 개발', '자율주행 로봇 Motion Control & Path Planning 개발', 'Sensor Fusion Engineer (자율주행)', 'Machine Learning Engineer (자율주행)', '모빌리티 서비스 UX 디자이너 (자율주행/IVI)', '자율주행 무인 배달로봇 SLAM 위치추정(Localization) 개발자', '자율주행 Perception Engineer']
22


In [76]:
#주소가 떴으나, 스크롤 문제로 화면에 안떠 몇개의 값을 못가져오는 문제 해결
# 기업주소 리스트
address_list = []

# href_lists에 있는 각 링크에 대해 처리
for url in href_lists:
    # 페이지 이동
    driver.get(url)

    # 스크롤을 아래로 내리는 동작을 반복
    while True:
        # 현재 페이지의 HTML 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 기업 주소를 포함한 <section> 요소 찾기
        address_div = soup.find('section', class_='JobWorkPlace_className__ra6rp')

        if address_div:
            # <section> 내부의 주소 정보를 찾아서 리스트에 추가
            address_items = address_div.find_all('span', class_='body')
            address = [address_item.text.strip() for address_item in address_items]

            # URL과 해당하는 주소 리스트를 한 리스트에 추가
            address_list.append(address)
            break  # 주소를 찾았으므로 루프 종료

        # 스크롤을 크게 아래로 내림 (300픽셀씩)
        driver.execute_script("window.scrollBy(0, 300);")
        time.sleep(1)  # 스크롤 후 충분한 시간 대기



In [77]:
# address_list에 있는 각 기업주소에 대해 처리
for i in range(len(address_list)):
    # 첫 번째 요소를 제외한 나머지 주소로 갱신
    address_list[i] = address_list[i][1:]

# 리스트 출력
for address in address_list:
    print(f'{address}\n')

print(len(address_list))

['서울 강남구 언주로81길 13 진일빌딩 3층']

['서초구 강남대로 311, 19층']

['경기도 성남시 수정구 창업로 43, 판교 글로벌비즈센터 A동 907~910호']

['서울 강남구 언주로81길 13 진일빌딩 3층']

['안양시 동안구 엘에스로116번길 118 안양2차 SKV1 Center']

['서울시 성동구 아차산로17길 48']

['서초구 논현로 145, 수냐빌딩 3층']

['성남시 수정구 금토로80번길 37(금토동)']

['성남시 분당구 성남대로69 로드랜드EZ타워']

['서울 금천구 디지털로10길 78 가산테라타워 10층']

['서울 강남구 언주로81길 13 진일빌딩 3층']

['성남시 수정구 금토로80번길 37(금토동)']

['경기 성남시 분당구 벌말로 50번길 41 투아이센터 7층']

['성남시 분당구 성남대로925번길 16, 7층 701호 (야탑동)']

['경기도 성남시 분당구 판교로 228번길 17, 판교벤처밸리 2단지 1동 4층 긴트']

['서초구 강남대로 311, 19층']

['성남시 분당구 성남대로69 로드랜드EZ타워']

['서울특별시 강남구 남부순환로 2621 (도곡동, 서브원 강남빌딩) 9층']

['서울특별시 강남구 남부순환로 2621 (도곡동, 서브원 강남빌딩) 9층']

['서울특별시 강남구 남부순환로 2621 (도곡동, 서브원 강남빌딩) 9층']

['성남시 분당구 성남대로69 로드랜드EZ타워']

['성남시 분당구 성남대로69 로드랜드EZ타워']

22


In [78]:
#연봉(연봉데이터가 딱히 없어 태그 연봉 관련 내용 삽입)
salary_list = []

# href_lists에 있는 각 링크에 대해 처리
for url in href_lists:
    # 페이지 이동
    driver.get(url)


    while True:
        # 현재 페이지의 HTML 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 기술 스택을 포함한 <div> 요소 찾기
        salary_div = soup.find('div', class_='Tags_tagsClass__mvehZ')

        if salary_div:
            salary_items = salary_div.find_all('li')
            first_li = salary_items[0] if salary_items else None

            if first_li and "연봉" in first_li.text:
                salaries = [salary_item.text.strip().replace("#", "") for salary_item in salary_items]
                salary_list.append(salaries[0])
                break
            else:
                salary_list.append(["-"])
                break


     
# 리스트 출력
for i in salary_list:
    print(f'{i}\n')

print(len(salary_list))

연봉업계평균이상

['-']

['-']

연봉업계평균이상

연봉업계평균이상

['-']

['-']

연봉업계평균이상

['-']

['-']

연봉업계평균이상

연봉업계평균이상

['-']

연봉업계평균이상

연봉업계평균이상

['-']

['-']

연봉상위6~10%

연봉상위6~10%

연봉상위6~10%

['-']

['-']

22


In [89]:
# 자격요건 텍스트를 저장할 리스트
qualifications_list = []

# href_lists에 있는 각 링크에 대해 처리
for url in href_lists:
    driver.get(url)

    # 스크롤을 아래로 내리는 동작을 반복
    while True:
        # 현재 페이지의 HTML 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # "자격요건" 부분을 추출
        qualifications_header = soup.find('h3', string='자격요건')

        if qualifications_header:
            # "자격요건" 부분이 포함된 <span> 요소 찾기
            qualifications_span = qualifications_header.find_next('p').find('span')

            if qualifications_span:
                # <span> 요소에서 텍스트 추출
                qualifications_text = qualifications_span.get_text(strip=True)
                # 리스트에 자격요건 추가
                qualifications_list.append(qualifications_text)
                break  # 자격요건을 찾았으므로 루프 종료
            else:
                print('자격요건이 포함된 <span> 요소를 찾을 수 없습니다.')
        else:
            # 스크롤을 계속해서 아래로 내림
            driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)

for qualifications in qualifications_list:
    print(f'{qualifications}\n')

print(len(qualifications_list))

[자격요건]• 학사 이상 (컴퓨터 공학/감성 및 인간공학 등 전공)• 프론트엔드 개발 경력 2년 이상• 프로그래밍 능력 (HTML, CSS, JavaScript, etc.)[채용절차]• 서류전형   실무·임원면접  채용검진  최종합격(상황에 따라 실무면접이 별도로 진행될 수 있습니다.)[제출서류]• 이력서, 자기소개서, 최종학력 성적증명서(90일 이내 발급), 포트폴리오(선택), 이력서 내 희망연봉 필수기재[근무조건]• 근무형태: 정규직• 근무일시: 주 5일(월~금), 시차출근제• 급여조건: 협의 후 결정

• 전자전기/컴퓨터공학 학사 이상 또는 이와 비슷한 동종업계 경력• C/C++ 또는 Python 개발• MS or higher degree in CS/CE/EE(Candidate) or equivalent industry experience• Experience in the field of deep learning/ machine learning• Strong programming skills in C/C++ or Python

• 관련 분야 학사학위(4년제) 이상, 혹은 유관 분야 경력 2년 이상이신 분• C/C++ 또는 Python 프로그래밍 언어를 익숙하게 다룰 수 있는  분• 딥러닝/기계학습 분야 경험

[자격요건]•  학사학위 이상(자동차공학, 기계공학, 로보틱스, 컴퓨터 공학, 전기전자 등 관련 분야 전공)•  C/C++, Python 등을 활용한 프로그래밍 프로젝트 경험•  (경력) 프로그래밍 능력 (C/C++, Python)•  (경력) 관련분야 경력 2년 이상•  운전 가능자[채용절차]• 서류전형   실무·임원면접  채용검진  최종합격(상황에 따라 실무면접이 별도로 진행될 수 있습니다.)[제출서류]• 이력서, 자기소개서, 최종학력 성적증명서(90일 이내 발급), 포트폴리오(선택), 이력서 내 희망연봉 필수기재[근무조건]• 근무형태: 정규직• 근무일시: 주 5일(월~금), 시차출근제• 급여조건: 협의 후 결정

- 2년이상 경력자- C++ 혹

In [84]:
#담당업무
work_list = []

# href_lists에 있는 각 링크에 대해 처리
for url in href_lists:
    driver.get(url)

    # 스크롤을 아래로 내리는 동작을 반복
    while True:
        # 현재 페이지의 HTML 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # "주요업무" 부분을 추출
        work_header = soup.find('h3', string='주요업무')

        if work_header:
            work_span = work_header.find_next('p').find('span')

            if work_span:
                # <span> 요소에서 텍스트 추출
                work_text = work_span.get_text(strip=True)
                work_list.append(work_text)
                break  
            else:
                print('담당업무가 포함된 <span> 요소를 찾을 수 없습니다.')
        else:
            # 스크롤을 계속해서 아래로 내림
            driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)

for work in work_list:
    print(f'{work}\n')

print(len(work_list))

• 자율주행관련 UI/UX 컨셉 기획- 자율주행 Driving Experience에 적합한 컨셉 기획- 자율주행 UI 프로토타입 설계• 자율주행 프론트엔드 개발- 자율주행시 탑승자와 시스템간 UI 구현- 자율주행 인터페이스 유지 및 보수

• 라이다 인지 기술 개발 (C/Python)• 자율주행 인지 솔루션 연구, 디자인, 개발• 실시간 데이터를 활용한 차량주변 객체 움직임 예측• Develop LiDAR perception algorithm (C/Python)• Research, design, and develop perception solutions and algorithms• Implement techniques in object classification, tracking, and prediction• Use real-time estimation from surroundings to predict the motion of surrounding vehicles

• 영상처리 기술 분석 및 구현• AI 기반 실시간 객체 인지, 추적 등의 기술 구현• 관련 논문 게재 또는 특허 출원

• 자율주행 소프트웨어 개발 및 최적화 (아래 중 1)- 딥러닝 기반 인식 알고리즘 개발- 다중 센서 기반 정밀 측위 알고리즘 개발- 동적 객체 경로 예측 알고리즘 개발- 경로생성 알고리즘 개발- 동역학 기반 차량 제어 알고리즘 개발- 시뮬레이션 환경 구축 및 알고리즘 최적화

사용기술- ROS, Python, Github, Git, 리눅스 등주요업무- 자율주행차량 제어 프로그램 개발- 자사 측위 알고리즘 개선 및 엔진 고도화- 라이다, 카메라, BLE 등의 다양한 이동체 탑재 센서 기반 측위 알고리즘 개발- 새로운 기술개발 R&D 사업 기획, 시장 조사

[팀 소개]XYZ는 자율주행 로봇을 전문적으로 개발하는 스타트업으로 자율주행 로봇을 활용하여 건축물 내 XY축 이동뿐만 아니라 Z축 이동을 통해 다양한 식음료를 전달하고 있습니다.XYZ의 로봇 자동화팀은 로봇 서비스 운영과 

In [90]:
#우대사항
preferential_list = []

# href_lists에 있는 각 링크에 대해 처리
for url in href_lists:
    driver.get(url)

    # 스크롤을 아래로 내리는 동작을 반복
    while True:
        # 현재 페이지의 HTML 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # "주요업무" 부분을 추출
        preferential_header = soup.find('h3', string='우대사항')

        if preferential_header:
            preferential_span = preferential_header.find_next('p').find('span')

            if preferential_span:
                # <span> 요소에서 텍스트 추출
                preferential_text = preferential_span.get_text(strip=True)
                preferential_list.append(preferential_text)
                break  
            else:
                print('우대사항이 포함된 <span> 요소를 찾을 수 없습니다.')
        else:
            # 스크롤을 계속해서 아래로 내림
            driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)

for preferential in preferential_list:
    print(f'{preferential}\n')

print(len(preferential_list))

•  IT기기, 자동차 등 제조업 기반 UX/UI 디자인 경력 보유자•  2D/3D 디자인 툴 및 프로토타입 툴 직무 경험 보유자•  웹 어플리케이션 디자인/설계 경험•  Linux 상 프로그램 개발 능력

• 전자전기/컴퓨터 공학관련 석사 이상학위• 라이다/비전센서 인지(분류 등) 관련 프로젝트 경험• ADAS/Autonomous Driving 관련 업계 경험• MS in CS/CE/EE field(Especially computer vision, digital signal processing)• Experience in autonomous driving competition or development• Experience LiDAR / Vision sensor-related project(deep learning/machine learning)• Experience in deploying perception algorithms

• 관련 분야 석사 학위 이상 소지자• 자율주행 시스템 구성에 대한 이해• 자율주행 센서(GNSS, IMU, Camera, Lidar 등) 사용 경험• 관련(자율주행 및 AI) 분야 저서/학술활동 이력• 자율주행 및 로보틱스 관련 프로젝트 참여 혹은 개발 경험• 자율주행 차량 혹은 로봇을 운용해본 경험• 자유롭게 자신의 의견을 제시할 수 있으며, 도전 정신이 있으신 분

• 자율주행 프로젝트 수행 경험 보유자• 자율주행 시스템에 대한 전반적인 이해• 관련분야 석/박사 학위

- 통계 및 수학 전공자- 센서 및 GPS 신호분석, 응용 알고리즘 개발 이해도 보유- Embedded SW개발 (C / C++)- 데이터 분석 (Matlab 또는 python)- ROS에 대한 이해- 측위기술 과제 경험, Embedded 시스템 구현경험- 머신러닝기반 알고리즘 개발 경험- 자율주행 시스템의 전반적인 구조에 대한 이해- SLAM, Kalman filter, 차량 동력학(Vehicle dynamics)에 대한 이해

- 자동화 시스템 제어 경험 보

In [91]:
driver.quit()

In [92]:
# 같은 인덱스끼리 합치기(자격요건 + 우대사항 + 담당업무)
required_list = [qual + pref + work for qual, pref, work in zip(qualifications_list, preferential_list, work_list)]

for item in required_list:
    print(item)


[자격요건]• 학사 이상 (컴퓨터 공학/감성 및 인간공학 등 전공)• 프론트엔드 개발 경력 2년 이상• 프로그래밍 능력 (HTML, CSS, JavaScript, etc.)[채용절차]• 서류전형   실무·임원면접  채용검진  최종합격(상황에 따라 실무면접이 별도로 진행될 수 있습니다.)[제출서류]• 이력서, 자기소개서, 최종학력 성적증명서(90일 이내 발급), 포트폴리오(선택), 이력서 내 희망연봉 필수기재[근무조건]• 근무형태: 정규직• 근무일시: 주 5일(월~금), 시차출근제• 급여조건: 협의 후 결정•  IT기기, 자동차 등 제조업 기반 UX/UI 디자인 경력 보유자•  2D/3D 디자인 툴 및 프로토타입 툴 직무 경험 보유자•  웹 어플리케이션 디자인/설계 경험•  Linux 상 프로그램 개발 능력• 자율주행관련 UI/UX 컨셉 기획- 자율주행 Driving Experience에 적합한 컨셉 기획- 자율주행 UI 프로토타입 설계• 자율주행 프론트엔드 개발- 자율주행시 탑승자와 시스템간 UI 구현- 자율주행 인터페이스 유지 및 보수
• 전자전기/컴퓨터공학 학사 이상 또는 이와 비슷한 동종업계 경력• C/C++ 또는 Python 개발• MS or higher degree in CS/CE/EE(Candidate) or equivalent industry experience• Experience in the field of deep learning/ machine learning• Strong programming skills in C/C++ or Python• 전자전기/컴퓨터 공학관련 석사 이상학위• 라이다/비전센서 인지(분류 등) 관련 프로젝트 경험• ADAS/Autonomous Driving 관련 업계 경험• MS in CS/CE/EE field(Especially computer vision, digital signal processing)• Experience in autonomous driving competition or development• E

In [93]:
keyword_list = ["자율주행"] * len(company_names)

print(len(keyword_list))
print(keyword_list)

22
['자율주행', '자율주행', '자율주행', '자율주행', '자율주행', '자율주행', '자율주행', '자율주행', '자율주행', '자율주행', '자율주행', '자율주행', '자율주행', '자율주행', '자율주행', '자율주행', '자율주행', '자율주행', '자율주행', '자율주행', '자율주행', '자율주행']


In [94]:
import pandas as pd

# 데이터프레임 생성
data = {
    '기업이름': company_names,
    '연봉': salary_list,
    '기업주소': address_list,
    '키워드': keyword_list,
    '담당업무': job_titles,
    '필요역량': required_list
}

df = pd.DataFrame(data)

# 데이터프레임 출력
df.head()

,기업이름,연봉,기업주소,키워드,담당업무,필요역량
0,컨트롤웍스,연봉업계평균이상,[서울 강남구 언주로81길 13 진일빌딩 3층],자율주행,[2년 이상] 자율주행 Front-End 개발자,[자격요건]• 학사 이상 (컴퓨터 공학/감성 및 인간공학 등 전공)• 프론트엔드 개...
1,뷰런테크놀로지,[-],"[서초구 강남대로 311, 19층]",자율주행,자율주행 인지 개발자,• 전자전기/컴퓨터공학 학사 이상 또는 이와 비슷한 동종업계 경력• C/C++ 또는...
2,씨엔비스,[-],"[경기도 성남시 수정구 창업로 43, 판교 글로벌비즈센터 A동 907~910호]",자율주행,자율주행 인지 개발자,"• 관련 분야 학사학위(4년제) 이상, 혹은 유관 분야 경력 2년 이상이신 분• C..."
3,컨트롤웍스,연봉업계평균이상,[서울 강남구 언주로81길 13 진일빌딩 3층],자율주행,자율주행 SW 엔지니어,"[자격요건]• 학사학위 이상(자동차공학, 기계공학, 로보틱스, 컴퓨터 공학, 전기..."
4,베스텔라랩,연봉업계평균이상,[안양시 동안구 엘에스로116번길 118 안양2차 SKV1 Center],자율주행,자율주행 연구원(경기/대구)(전문연구요원가능),- 2년이상 경력자- C++ 혹은 Python 프로그래밍 능력- 국영문 학술논문 등...


In [95]:
# • 문자를 ,으로 바꾸기
df['담당업무'] = df['담당업무'].str.replace('•', ',')
df['필요역량'] = df['필요역량'].str.replace('•', ',')

# 작은 따옴표 및 괄호 제거
df['연봉'] = df['연봉'].astype(str).str.replace('[', '').str.replace(']', '').str.strip("'")
df['기업주소'] = df['기업주소'].astype(str).str.replace('[', '').str.replace(']', '').str.strip("'")
df.head()

,기업이름,연봉,기업주소,키워드,담당업무,필요역량
0,컨트롤웍스,연봉업계평균이상,서울 강남구 언주로81길 13 진일빌딩 3층,자율주행,[2년 이상] 자율주행 Front-End 개발자,"[자격요건], 학사 이상 (컴퓨터 공학/감성 및 인간공학 등 전공), 프론트엔드 개..."
1,뷰런테크놀로지,-,"서초구 강남대로 311, 19층",자율주행,자율주행 인지 개발자,", 전자전기/컴퓨터공학 학사 이상 또는 이와 비슷한 동종업계 경력, C/C++ 또는..."
2,씨엔비스,-,"경기도 성남시 수정구 창업로 43, 판교 글로벌비즈센터 A동 907~910호",자율주행,자율주행 인지 개발자,", 관련 분야 학사학위(4년제) 이상, 혹은 유관 분야 경력 2년 이상이신 분, C..."
3,컨트롤웍스,연봉업계평균이상,서울 강남구 언주로81길 13 진일빌딩 3층,자율주행,자율주행 SW 엔지니어,"[자격요건], 학사학위 이상(자동차공학, 기계공학, 로보틱스, 컴퓨터 공학, 전기..."
4,베스텔라랩,연봉업계평균이상,안양시 동안구 엘에스로116번길 118 안양2차 SKV1 Center,자율주행,자율주행 연구원(경기/대구)(전문연구요원가능),- 2년이상 경력자- C++ 혹은 Python 프로그래밍 능력- 국영문 학술논문 등...


In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   기업이름    22 non-null     object
 1   연봉      22 non-null     object
 2   기업주소    22 non-null     object
 3   키워드     22 non-null     object
 4   담당업무    22 non-null     object
 5   필요역량    22 non-null     object
dtypes: object(6)
memory usage: 1.2+ KB


In [97]:
import re

# 경력 정보 추출을 위한 정규표현식
career_pattern = re.compile(r'(\d+년.*?)')

# DataFrame의 각 행에 대해 경력 정보 추출 및 새로운 열에 추가
df['고용형태'] = df['필요역량'].apply(lambda x: career_pattern.findall(x)[0] if career_pattern.findall(x) else '-')

# 결과 출력
print(df[['고용형태']])

   고용형태
0    2년
1     -
2    4년
3    2년
4    2년
5    2년
6     -
7    1년
8     -
9    7년
10    -
11   3년
12   5년
13   2년
14   4년
15   2년
16    -
17   5년
18   5년
19   7년
20    -
21    -


In [98]:
df = df[['기업이름','고용형태','연봉','기업주소','키워드','담당업무','필요역량' ]]

In [99]:
df.head()

,기업이름,고용형태,연봉,기업주소,키워드,담당업무,필요역량
0,컨트롤웍스,2년,연봉업계평균이상,서울 강남구 언주로81길 13 진일빌딩 3층,자율주행,[2년 이상] 자율주행 Front-End 개발자,"[자격요건], 학사 이상 (컴퓨터 공학/감성 및 인간공학 등 전공), 프론트엔드 개..."
1,뷰런테크놀로지,-,-,"서초구 강남대로 311, 19층",자율주행,자율주행 인지 개발자,", 전자전기/컴퓨터공학 학사 이상 또는 이와 비슷한 동종업계 경력, C/C++ 또는..."
2,씨엔비스,4년,-,"경기도 성남시 수정구 창업로 43, 판교 글로벌비즈센터 A동 907~910호",자율주행,자율주행 인지 개발자,", 관련 분야 학사학위(4년제) 이상, 혹은 유관 분야 경력 2년 이상이신 분, C..."
3,컨트롤웍스,2년,연봉업계평균이상,서울 강남구 언주로81길 13 진일빌딩 3층,자율주행,자율주행 SW 엔지니어,"[자격요건], 학사학위 이상(자동차공학, 기계공학, 로보틱스, 컴퓨터 공학, 전기..."
4,베스텔라랩,2년,연봉업계평균이상,안양시 동안구 엘에스로116번길 118 안양2차 SKV1 Center,자율주행,자율주행 연구원(경기/대구)(전문연구요원가능),- 2년이상 경력자- C++ 혹은 Python 프로그래밍 능력- 국영문 학술논문 등...


In [100]:
import csv

df.to_csv('output.csv', index=False)